# Centralized Solution of Multi-Regional Transmission Expansion Horizontal Investment Coordination Problem

This notebook contains code for a three zone test case. The case consists of an artificial merging of IEEE 14 nodes, IEEE 30 nodes, and a 5 node test system, respectively, for each zone. This notebook assumes there is a single entity (e.g. national planner) to solve the optimization problem centrally.
Results from this notebook serve a benchmark for the decentralized algorithms. 

In [1]:
# activate julia environment in a controlled way
julia_environment = "../central_control_PyJuMP/Julia_src/activate.jl"
include(julia_environment)

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.t

Activating the Julia virtual environment


No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [3]:
# activate required packages
using Pkg
using JuMP
using GLPK
using HiGHS
using Gurobi
#using CPLEX
#using Xpress
#using Cbc
#using Clp
using XLSX
using CSV
using DataFrames
using Random
#using PrettyPrint

# print package status
Pkg.status()

Status `~/.julia/environments/v1.5/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [336ed68f] CSV v0.8.5
  [9961bab8] Cbc v0.8.1
  [e2554f3b] Clp v0.8.4
  [a93c6f00] DataFrames v0.22.7
  [864edb3b] DataStructures v0.18.13
  [60bf3e95] GLPK v0.14.14
  [5c1252a2] GeometryBasics v0.3.13
  [2e9cd046] Gurobi v0.9.14
  [87dc4568] HiGHS v0.2.3
  [7073ff75] IJulia v1.23.1
  [b6b21f68] Ipopt v0.7.0
  [4076af6c] JuMP v0.21.10
  [b8f27783] MathOptInterface v0.9.22
  [fdba3010] MathProgBase v0.7.8
  [91a5bcdd] Plots v1.15.2
  [e690365d] PowerSimulations v0.13.0
  [bcd98974] PowerSystems v1.3.3
  [438e738f] PyCall v1.95.1
  [731186ca] RecursiveArrayTools v2.20.0
  [82193955] SCIP v0.9.8
  [2913bbd2] StatsBase v0.33.21
  [fdbf4ff8] XLSX v0.9.0
  [ddb6d928] YAML v0.4.8
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics


## Problem set up

Set up the example problem by reading the input data files from the `../Input_Data/` folder, then separate the data frames for each zone.

The input data includes: 

* `CandLine.csv` which contains information on shared _candidate lines_ between zones
* `CandLineInt.csv` which contains information on internal _candidate lines_ within each zone
* `SharedEline.csv` which contains information on existing _shared lines_ between zones
* `Tran.csv` which contains information on existing transmission lines within each zone (i.e. as defined in the cited IEEE test systems)
* `Gen.csv` which contains information on the generators as defined in the cited IEEE test systems
* `Load.csv` which contains information on the loads as defined in the cited IEEE test systems

After loading the input data, construct several DataFrames for each zone, i.e. for each set of lines, generators, loads, etc.


In [4]:
# Read input files to set the problem
data_path = "../Input_Data/"

# note that network data are convertered xlsx data (not true csv files)
shared_cand = DataFrame(XLSX.readtable(data_path * "CandLine.csv", "Taul1", header=true)) # shared candidate lines
int_cand =  DataFrame(XLSX.readtable(data_path * "CandLineInt.csv", "Taul1", header=true)) # internal candidate lines
shared_ex =  DataFrame(XLSX.readtable(data_path * "SharedEline.csv", "Taul1", header=true)) # shared existing lines
int_ex = DataFrame(XLSX.readtable(data_path * "Tran.csv", "Taul1", header=true)) # internal existing lines
gen =  DataFrame(XLSX.readtable(data_path * "Gen.csv" , "Taul1", header=true)) # generator data
load =  DataFrame(XLSX.readtable(data_path * "Load.csv", "Taul1", header=true)) # load data

# read scenario and zone summary data. These are true CSV files, not XLSX
scen_prob = CSV.read(data_path * "Scenario_Probability.csv", DataFrame) # scenario probabilities
zone_summary = CSV.read(data_path * "Zone_Summary.csv", DataFrame) # region-number of nodes

,Zone,Nodes_Total
,Int64,Int64
1,1,14
2,2,30
3,3,5


In [5]:
Random.seed!(123)
for i in 1:size(load,1)
    load.P_load1[i] = load.P_load1[i] * rand(24)
    load.P_load2[i] = load.P_load2[i] * rand(24)
    load.P_load3[i] = load.P_load3[i] * rand(24)
    load.P_load4[i] = load.P_load4[i] * rand(24)
end

In [6]:
# Functionally define dataframes for each element (loads, generators, etc.) within each zone
# The left hand side is the singleton function that references the dataframe on the right hand side, e.g. l(1) is the load dataframe for zone 1


l(i,s) = load[load.zoneNum .== i, [1,2,s+2]] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1.* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i
scen_weight(s) = scen_prob.scen_weight[scen_prob.scenario .== s,:]

scen_weight (generic function with 1 method)

## Build the Optimization Problem in JuMP

Initialize the JuMP model object by calling `Model(ENGING.Optimizer)` where `ENGINE` is your optimization engine of choice, e.g. Gurobi, CPLEX, Xpress, HiGHS, etc. The optimization engine must have a solver for mixed integer linear programing (MIP). 

After initializing the model, we will add the variables, constraints, and objective function. 
Decision variables for the model (and their variable type) include:
> * Generator dispatch (float)
> * Decision on candidate shared lines (binary)
> * Decision on candidate internal lines (binary)
> * Power flow (float) and phase angle (float) on candidate shared lines
> * Power flow (float) and phase angle (float) on candidate internal lines
> * Power flow (float) and phase angle (float) on existing shared lines
> * Power flow (float) and phase angle (float) on existing internal lines

The objective function is the `total_cost` to build new lines and dispatch generation. The cost is defined as:
> \[EQUATION GOES HERE\]

The constraints include: 
> \[EQUATIONS GO HERE\]

In [7]:
# Initialize the JuMP model object and set the optimization engine
# solvers = [Gurobi.Optimizer, Clp.Optimzer, Cbc.Optimizer, HiGHS.Optimizer, Ipopt.Optimizer, CPLEX.Optimizer, Xpress.Optimizer]

Mod = Model(Gurobi.Optimizer)

Academic license - for non-commercial use only - expires 2023-09-07


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [8]:

zone_numbeddr = size(zone_summary,1)

3

In [9]:
Tmax = 24

24

In [10]:
# Add the decision variables to the model

# Decision variables for the power generation at each node
@variable(Mod,0 <= gen_var_1[1:14,1:nrow(scen_prob),1:Tmax]) # generation at node 1
@variable(Mod,0 <= gen_var_2[1:30,1:nrow(scen_prob),1:Tmax]) # generation at node 2
@variable(Mod,0 <= gen_var_3[1:5,1:nrow(scen_prob),1:Tmax]) # generation at node 3

# Decision variables for the candidate lines build/no build decision
@variable(Mod,shared_line_decision_var[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines
@variable(Mod,int_line_decision_var_1[1:nrow(int_c(1))], Bin) #Decision variable for internal candidate lines of zone 1 (Binary)
@variable(Mod,int_line_decision_var_2[1:nrow(int_c(2))], Bin) #Decision variable for internal candidate lines of zone 2 (Binary)
@variable(Mod,int_line_decision_var_3[1:nrow(int_c(3))], Bin) #Decision variable for internal candidate lines of zone 3 (Binary)

# Decision variables for the power flow on the candidate lines
@variable(Mod, shared_cand_flow[1:nrow(shared_cand),1:nrow(scen_prob),1:Tmax])  #Power flowing on shared candidate lines 
@variable(Mod, int_cand_flow_1[1:nrow(int_c(1)),1:nrow(scen_prob),1:Tmax]) #Power flowing on internal candidate lines of zone 1 
@variable(Mod, int_cand_flow_2[1:nrow(int_c(2)),1:nrow(scen_prob),1:Tmax]) #Power flowing on internal candidate lines of zone 2
@variable(Mod, int_cand_flow_3[1:nrow(int_c(3)),1:nrow(scen_prob),1:Tmax]) #Power flowing on internal candidate lines of zone 3

# Decision variables for the phase angle of the candidate lines
@variable(Mod, 0 <= node_voltage_phase_angle[i=1:nrow(zone_summary),j=1:zone_summary.Nodes_Total[i], 1:nrow(scen_prob),1:Tmax]<= 2*pi)

# Decision variables for the power flow on the existing lines
@variable(Mod, shared_ex_flow[1:nrow(shared_ex),1:nrow(scen_prob),1:Tmax])  #Power flowing on shared existing shared lines

@variable(Mod, int_ex_flow_1[1:nrow(int_e(1)),1:nrow(scen_prob),1:Tmax])  #Power flowing on internal existing lines of zone 1 
@variable(Mod, int_ex_flow_2[1:nrow(int_e(2)),1:nrow(scen_prob),1:Tmax])  #Power flowing on internal existing lines of zone 2
@variable(Mod, int_ex_flow_3[1:nrow(int_e(3)),1:nrow(scen_prob),1:Tmax])  #Power flowing on internal existing lines of zone 3


7×4×24 Array{VariableRef,3}:
[:, :, 1] =
 int_ex_flow_3[1,1,1]  int_ex_flow_3[1,2,1]  …  int_ex_flow_3[1,4,1]
 int_ex_flow_3[2,1,1]  int_ex_flow_3[2,2,1]     int_ex_flow_3[2,4,1]
 int_ex_flow_3[3,1,1]  int_ex_flow_3[3,2,1]     int_ex_flow_3[3,4,1]
 int_ex_flow_3[4,1,1]  int_ex_flow_3[4,2,1]     int_ex_flow_3[4,4,1]
 int_ex_flow_3[5,1,1]  int_ex_flow_3[5,2,1]     int_ex_flow_3[5,4,1]
 int_ex_flow_3[6,1,1]  int_ex_flow_3[6,2,1]  …  int_ex_flow_3[6,4,1]
 int_ex_flow_3[7,1,1]  int_ex_flow_3[7,2,1]     int_ex_flow_3[7,4,1]

[:, :, 2] =
 int_ex_flow_3[1,1,2]  int_ex_flow_3[1,2,2]  …  int_ex_flow_3[1,4,2]
 int_ex_flow_3[2,1,2]  int_ex_flow_3[2,2,2]     int_ex_flow_3[2,4,2]
 int_ex_flow_3[3,1,2]  int_ex_flow_3[3,2,2]     int_ex_flow_3[3,4,2]
 int_ex_flow_3[4,1,2]  int_ex_flow_3[4,2,2]     int_ex_flow_3[4,4,2]
 int_ex_flow_3[5,1,2]  int_ex_flow_3[5,2,2]     int_ex_flow_3[5,4,2]
 int_ex_flow_3[6,1,2]  int_ex_flow_3[6,2,2]  …  int_ex_flow_3[6,4,2]
 int_ex_flow_3[7,1,2]  int_ex_flow_3[7,2,2]     i

In [11]:
# Build the total_cost objective function one element and one zone at a time

# Initialize total_cost function
@expression(Mod,total_cost,0)

0

In [12]:
# Define the scenario weighted gen cost for zone 1
@expression(Mod, gen_costNS1[n=1:14,s=1:nrow(scen_prob),t=1:Tmax], (gen_var_1[n,s,t] .* sum((g(1).gNodeID .== n) .* MC(1))))
@expression(Mod, gen_cost_prob_weightedS1[s=1:nrow(scen_prob),t=1:Tmax], (scen_weight(s)[1]).*sum(gen_costNS1[n,s,t] for n in 1:14))

# Add the expectation of gen cost for zone 1 to the total_cost function
@expression(Mod, gen_cost_expected1, sum(sum(gen_cost_prob_weightedS1[s,t] for s in 1:nrow(scen_prob)) for t in 1:Tmax))
Mod[:total_cost] += gen_cost_expected1


8.57573483 gen_var_1[1,1,1] + 13.75 gen_var_1[2,1,1] + 8.57573483 gen_var_1[1,2,1] + 13.75 gen_var_1[2,2,1] + 8.57573483 gen_var_1[1,3,1] + 13.75 gen_var_1[2,3,1] + 8.57573483 gen_var_1[1,4,1] + 13.75 gen_var_1[2,4,1] + 8.57573483 gen_var_1[1,1,2] + 13.75 gen_var_1[2,1,2] + 8.57573483 gen_var_1[1,2,2] + 13.75 gen_var_1[2,2,2] + 8.57573483 gen_var_1[1,3,2] + 13.75 gen_var_1[2,3,2] + 8.57573483 gen_var_1[1,4,2] + 13.75 gen_var_1[2,4,2] + 8.57573483 gen_var_1[1,1,3] + 13.75 gen_var_1[2,1,3] + 8.57573483 gen_var_1[1,2,3] + 13.75 gen_var_1[2,2,3] + 8.57573483 gen_var_1[1,3,3] + 13.75 gen_var_1[2,3,3] + 8.57573483 gen_var_1[1,4,3] + 13.75 gen_var_1[2,4,3] + 8.57573483 gen_var_1[1,1,4] + 13.75 gen_var_1[2,1,4] + 8.57573483 gen_var_1[1,2,4] + 13.75 gen_var_1[2,2,4] + 8.57573483 gen_var_1[1,3,4] + 13.75 gen_var_1[2,3,4] + 8.57573483 gen_var_1[1,4,4] + 13.75 gen_var_1[2,4,4] + 8.57573483 gen_var_1[1,1,5] + 13.75 gen_var_1[2,1,5] + 8.57573483 gen_var_1[1,2,5] + 13.75 gen_var_1[2,2,5] + 8.57573483

In [13]:
# Define the scenario weighted gen cost for zone 2
@expression(Mod, gen_costNS2[n=1:30,s=1:nrow(scen_prob),t=1:Tmax], (gen_var_2[n,s,t] .* sum((g(2).gNodeID .== n) .* MC(2))))
@expression(Mod, gen_cost_prob_weightedS2[s=1:nrow(scen_prob),t=1:Tmax], (scen_weight(s)[1]).*sum(gen_costNS2[n,s,t] for n in 1:30))

# Add the expectation of gen cost for zone 2 to the total_cost function
@expression(Mod, gen_cost_expected2, sum(sum(gen_cost_prob_weightedS2[s,t] for s in 1:nrow(scen_prob)) for t in 1:Tmax))
Mod[:total_cost]+=gen_cost_expected2

8.57573483 gen_var_1[1,1,1] + 13.75 gen_var_1[2,1,1] + 8.57573483 gen_var_1[1,2,1] + 13.75 gen_var_1[2,2,1] + 8.57573483 gen_var_1[1,3,1] + 13.75 gen_var_1[2,3,1] + 8.57573483 gen_var_1[1,4,1] + 13.75 gen_var_1[2,4,1] + 8.57573483 gen_var_1[1,1,2] + 13.75 gen_var_1[2,1,2] + 8.57573483 gen_var_1[1,2,2] + 13.75 gen_var_1[2,2,2] + 8.57573483 gen_var_1[1,3,2] + 13.75 gen_var_1[2,3,2] + 8.57573483 gen_var_1[1,4,2] + 13.75 gen_var_1[2,4,2] + 8.57573483 gen_var_1[1,1,3] + 13.75 gen_var_1[2,1,3] + 8.57573483 gen_var_1[1,2,3] + 13.75 gen_var_1[2,2,3] + 8.57573483 gen_var_1[1,3,3] + 13.75 gen_var_1[2,3,3] + 8.57573483 gen_var_1[1,4,3] + 13.75 gen_var_1[2,4,3] + 8.57573483 gen_var_1[1,1,4] + 13.75 gen_var_1[2,1,4] + 8.57573483 gen_var_1[1,2,4] + 13.75 gen_var_1[2,2,4] + 8.57573483 gen_var_1[1,3,4] + 13.75 gen_var_1[2,3,4] + 8.57573483 gen_var_1[1,4,4] + 13.75 gen_var_1[2,4,4] + 8.57573483 gen_var_1[1,1,5] + 13.75 gen_var_1[2,1,5] + 8.57573483 gen_var_1[1,2,5] + 13.75 gen_var_1[2,2,5] + 8.57573483

In [14]:
# Define the scenario weighted gen cost for zone 3
@expression(Mod, gen_costNS3[n=1:5,s=1:nrow(scen_prob),t=1:Tmax], (gen_var_3[n,s,t] .* sum((g(3).gNodeID .== n) .* MC(3))))
@expression(Mod, gen_cost_prob_weightedS3[s=1:nrow(scen_prob),t=1:Tmax], (scen_weight(s)[1]).*sum(gen_costNS3[n,s,t] for n in 1:5))

# Add the expectation of gen cost for zone 3 to the total_cost function
@expression(Mod, gen_cost_expected3, sum(sum(gen_cost_prob_weightedS3[s,t] for s in 1:nrow(scen_prob)) for t in 1:Tmax))
Mod[:total_cost]+=gen_cost_expected3

8.57573483 gen_var_1[1,1,1] + 13.75 gen_var_1[2,1,1] + 8.57573483 gen_var_1[1,2,1] + 13.75 gen_var_1[2,2,1] + 8.57573483 gen_var_1[1,3,1] + 13.75 gen_var_1[2,3,1] + 8.57573483 gen_var_1[1,4,1] + 13.75 gen_var_1[2,4,1] + 8.57573483 gen_var_1[1,1,2] + 13.75 gen_var_1[2,1,2] + 8.57573483 gen_var_1[1,2,2] + 13.75 gen_var_1[2,2,2] + 8.57573483 gen_var_1[1,3,2] + 13.75 gen_var_1[2,3,2] + 8.57573483 gen_var_1[1,4,2] + 13.75 gen_var_1[2,4,2] + 8.57573483 gen_var_1[1,1,3] + 13.75 gen_var_1[2,1,3] + 8.57573483 gen_var_1[1,2,3] + 13.75 gen_var_1[2,2,3] + 8.57573483 gen_var_1[1,3,3] + 13.75 gen_var_1[2,3,3] + 8.57573483 gen_var_1[1,4,3] + 13.75 gen_var_1[2,4,3] + 8.57573483 gen_var_1[1,1,4] + 13.75 gen_var_1[2,1,4] + 8.57573483 gen_var_1[1,2,4] + 13.75 gen_var_1[2,2,4] + 8.57573483 gen_var_1[1,3,4] + 13.75 gen_var_1[2,3,4] + 8.57573483 gen_var_1[1,4,4] + 13.75 gen_var_1[2,4,4] + 8.57573483 gen_var_1[1,1,5] + 13.75 gen_var_1[2,1,5] + 8.57573483 gen_var_1[1,2,5] + 13.75 gen_var_1[2,2,5] + 8.57573483

In [15]:
# Add  the investment cost terms to the total_cost function
@expression(Mod, investment_cost , (sum(shared_line_decision_var[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
            .+ sum(int_line_decision_var_1[c] .* int_c(1).costPerCap[c] .* int_c(1).interestRate[c] 
            .*((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c]) ./ (((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c])-1) for c in 1:nrow(int_c(1)))
            .+ sum(int_line_decision_var_2[c] .* int_c(2).costPerCap[c] .* int_c(2).interestRate[c] 
            .*((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c]) ./ (((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c])-1) for c in 1:nrow(int_c(2)))
            .+ sum(int_line_decision_var_3[c] .* int_c(3).costPerCap[c] .* int_c(3).interestRate[c] 
            .*((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c]) ./ (((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c])-1) for c in 1:nrow(int_c(3)))))
Mod[:total_cost]+=investment_cost

8.57573483 gen_var_1[1,1,1] + 13.75 gen_var_1[2,1,1] + 8.57573483 gen_var_1[1,2,1] + 13.75 gen_var_1[2,2,1] + 8.57573483 gen_var_1[1,3,1] + 13.75 gen_var_1[2,3,1] + 8.57573483 gen_var_1[1,4,1] + 13.75 gen_var_1[2,4,1] + 8.57573483 gen_var_1[1,1,2] + 13.75 gen_var_1[2,1,2] + 8.57573483 gen_var_1[1,2,2] + 13.75 gen_var_1[2,2,2] + 8.57573483 gen_var_1[1,3,2] + 13.75 gen_var_1[2,3,2] + 8.57573483 gen_var_1[1,4,2] + 13.75 gen_var_1[2,4,2] + 8.57573483 gen_var_1[1,1,3] + 13.75 gen_var_1[2,1,3] + 8.57573483 gen_var_1[1,2,3] + 13.75 gen_var_1[2,2,3] + 8.57573483 gen_var_1[1,3,3] + 13.75 gen_var_1[2,3,3] + 8.57573483 gen_var_1[1,4,3] + 13.75 gen_var_1[2,4,3] + 8.57573483 gen_var_1[1,1,4] + 13.75 gen_var_1[2,1,4] + 8.57573483 gen_var_1[1,2,4] + 13.75 gen_var_1[2,2,4] + 8.57573483 gen_var_1[1,3,4] + 13.75 gen_var_1[2,3,4] + 8.57573483 gen_var_1[1,4,4] + 13.75 gen_var_1[2,4,4] + 8.57573483 gen_var_1[1,1,5] + 13.75 gen_var_1[2,1,5] + 8.57573483 gen_var_1[1,2,5] + 13.75 gen_var_1[2,2,5] + 8.57573483

In [16]:
# Define constraints for each zone

# Define constraints for Zone 1
# This includes nodal power balance constraints and upper and lower bound of connecting generators
for n in 1:14
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            # Power balance constraint for each node
            @constraint(Mod, sum(g(1).gNodeID .== n) .* gen_var_1[n,s,t] .+ sum(l(1,s)[:,3].* (l(1,s).lNodeID .== n))[t]./100 .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(1) .* shared_cand_flow[:,s,t]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(1) .* shared_cand_flow[:,s,t]) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(1) .* shared_ex_flow[:,s,t]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(1) .* shared_ex_flow[:,s,t]) .+
            sum((int_c(1).tNodeID1 .== n) .* int_cand_flow_1[:,s,t]) .- sum((int_c(1).tNodeID2 .== n) .* int_cand_flow_1[:,s,t]) .+
            sum((int_e(1).tNodeID1 .== n) .* int_ex_flow_1[:,s,t]) .- sum((int_e(1).tNodeID2 .== n) .* int_ex_flow_1[:,s,t]))
            #Lower limit for generation of each node
            @constraint(Mod, sum(g(1).gNodeID .== n) .* gen_var_1[n,s,t] .<= sum((g(1).gNodeID .== n) .* g(1).PgMax)./100)
            #Upper limit for generation of each node
            @constraint(Mod, sum((g(1).gNodeID .== n) .* g(1).PgMin)./100 .<= sum(g(1).gNodeID .== n) .* gen_var_1[n,s,t])
        end
    end
end

In [17]:
# Define constraints for Zone 1
# This includes nodal power balance constraints and upper and lower bound of connecting generators

for n in 1:30
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            # Power balance constraint for each node
            @constraint(Mod, sum(g(2).gNodeID .== n) .* gen_var_2[n,s,t] .+ sum(l(2,s)[:,3] .* (l(2,s).lNodeID .== n))[t]./100 .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(2) .* shared_cand_flow[:,s,t]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(2) .* shared_cand_flow[:,s,t]) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(2) .* shared_ex_flow[:,s,t]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(2) .* shared_ex_flow[:,s,t]) .+
            sum((int_c(2).tNodeID1 .== n) .* int_cand_flow_2[:,s,t]) .- sum((int_c(2).tNodeID2 .== n) .* int_cand_flow_2[:,s,t]) .+
            sum((int_e(2).tNodeID1 .== n) .* int_ex_flow_2[:,s,t]) .- sum((int_e(2).tNodeID2 .== n) .* int_ex_flow_2[:,s,t]))
            #Lower limit for generation of each node
            @constraint(Mod, sum(g(2).gNodeID .== n) .* gen_var_2[n,s,t] .<= sum((g(2).gNodeID .== n) .* g(2).PgMax)./100)
            #Upper limit for generation of each node
            @constraint(Mod, sum((g(2).gNodeID .== n) .* g(2).PgMin)./100 .<= sum(g(2).gNodeID .== n) .* gen_var_2[n,s,t])
        end
    end
end

In [18]:
# Define constraints for Zone 1
# This includes nodal power balance constraints and upper and lower bound of connecting generators

for n in 1:5
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            # Power balance constraint for each node
            @constraint(Mod, sum(g(3).gNodeID .== n) .* gen_var_3[n,s,t] .+ sum(l(3,s)[:,3] .* (l(3,s).lNodeID .== n))[t]./100 .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(3) .* shared_cand_flow[:,s,t]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(3) .* shared_cand_flow[:,s,t]) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(3) .* shared_ex_flow[:,s,t]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(3) .* shared_ex_flow[:,s,t]) .+
            sum((int_c(3).tNodeID1 .== n) .* int_cand_flow_3[:,s,t]) .- sum((int_c(3).tNodeID2 .== n) .* int_cand_flow_3[:,s,t]) .+
            sum((int_e(3).tNodeID1 .== n) .* int_ex_flow_3[:,s,t]) .- sum((int_e(3).tNodeID2 .== n) .* int_ex_flow_3[:,s,t]))
            #Lower limit for generation of each node
            @constraint(Mod, sum(g(3).gNodeID .== n) .* gen_var_3[n,s,t] .<= sum((g(3).gNodeID .== n) .* g(3).PgMax)./100)
            #Upper limit for generation of each node
            @constraint(Mod, sum((g(3).gNodeID .== n) .* g(3).PgMin)./100 .<= sum(g(3).gNodeID .== n) .* gen_var_3[n,s,t])
        end
    end
end

In [19]:
# Deine constraints for the shared candidate lines
# The parameter big M is used to ensure equality constraints are satisfied when the candidate line is selected. 
# Choosing M is an art with some hueristic approaches. We want the smallest M that is still large enough to ensure the constraints are satisfied.
# For this problem, M should be related to the maximum power flow on the candidate line, assuming a 100 MW base power 
base_power = 100  # base power in MW
M = 500/base_power # M is 5 times the maximum power flow on the candidate line

for c in 1:nrow(shared_cand)
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c,s,t] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s,t] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s,t])))
            @constraint(Mod, shared_cand_flow[c,s,t] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s,t] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s,t])) .<= M .* (1 .- shared_line_decision_var[c]))
            #limiting the upper bound of power flow flowing within candidate shared lines
            @constraint(Mod, shared_cand_flow[c,s,t] .<= shared_line_decision_var[c] .*shared_cand.ptMax[c]./100)
            #Limiting the lower bound of power flowing within the candidate shared lines
            @constraint(Mod, -(shared_cand.ptMax[c]./100) .* shared_line_decision_var[c] .<= shared_cand_flow[c,s,t])
        end
    end
end

In [20]:
# Define constraints for existing shared lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(shared_ex)
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod, shared_ex_flow[h,s,t] .== (1 ./ shared_ex.reacT[h]) .* (node_voltage_phase_angle[shared_ex.nodeZone1[h],shared_ex.tNodeID1[h],s,t] .- node_voltage_phase_angle[shared_ex.nodeZone2[h],shared_ex.tNodeID2[h],s,t]))
            @constraint(Mod, shared_ex_flow[h,s,t] .<= shared_ex. ptMax[h]./100)
            @constraint(Mod, -shared_ex.ptMax[h]./100 .<= shared_ex_flow[h,s,t])
        end
    end
end

In [21]:
# Define constraints for Zone 1 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(1))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod,-M .* (1 .- int_line_decision_var_1[c]) .<= int_cand_flow_1[c,s,t] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s,t])))
            @constraint(Mod, int_cand_flow_1[c,s,t] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s,t])) .<= M .* (1 .- int_line_decision_var_1[c]))
            @constraint(Mod, int_cand_flow_1[c,s,t] .<= int_line_decision_var_1[c] .* int_c(1).ptMax[c]./100)
            @constraint(Mod, -(int_c(1).ptMax[c]./100) .*int_line_decision_var_1[c] .<= int_cand_flow_1[c,s,t])
        end
    end
end

In [22]:
# Define constraints for Zone 2 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(2))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod,-M .* (1 .- int_line_decision_var_2[c]) .<= int_cand_flow_2[c,s,t] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s,t])))
            @constraint(Mod, int_cand_flow_2[c,s,t] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s,t])) .<= M .* (1 .- int_line_decision_var_2[c]))
            @constraint(Mod, int_cand_flow_2[c,s,t] .<= int_line_decision_var_2[c] .* int_c(2).ptMax[c]./100)
            @constraint(Mod, -(int_c(2).ptMax[c]./100) .*int_line_decision_var_2[c] .<= int_cand_flow_2[c,s,t])
        end
    end
end

In [23]:
# Define constraints for Zone 3 internal candidate lines
# These include power flow constraints and upper and lower bound of power flow on each line
# Note that we use the same parameter M as the shared candidate lines in this example. 
# In practice you may want to configure M differently for internal lines if they have different order of magnitude of power flow compared to shared lines

for c in 1:nrow(int_c(3))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod,-M .* (1 .- int_line_decision_var_3[c]) .<= int_cand_flow_3[c,s,t] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s,t])))
            @constraint(Mod, int_cand_flow_3[c,s,t] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s,t] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s,t])) .<= M .* (1 .- int_line_decision_var_3[c]))
            @constraint(Mod, int_cand_flow_3[c,s,t] .<= int_line_decision_var_3[c] .* int_c(3).ptMax[c]./100)
            @constraint(Mod, -(int_c(3).ptMax[c]./100) .*int_line_decision_var_3[c] .<= int_cand_flow_3[c,s,t])
        end
    end
end

In [24]:
# Define constraints for Zone 1 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(1))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod, int_ex_flow_1[h,s,t] .== (1 ./ int_e(1).reacT[h]) .* (node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID1[h],s,t] .- node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID2[h],s,t]))
            @constraint(Mod, int_ex_flow_1[h,s,t] .<= int_e(1).ptMax[h]./100)
            @constraint(Mod, -int_e(1).ptMax[h]./100 .<= int_ex_flow_1[h,s,t])
        end
    end
end

In [25]:
# Define constraints for Zone 2 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(2))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod, int_ex_flow_2[h,s,t] .== (1 ./ int_e(2).reacT[h]) .* (node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID1[h],s,t] .- node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID2[h],s,t]))
            @constraint(Mod, int_ex_flow_2[h,s,t] .<= int_e(2).ptMax[h]./100)
            @constraint(Mod, -int_e(2).ptMax[h]./100 .<= int_ex_flow_2[h,s,t])
        end
    end
end

In [26]:
# Define constraints for Zone 3 internal existing lines
# These include power flow constraints and upper and lower bound of power flow on each line

for h in 1:nrow(int_e(3))
    for t in 1:Tmax
        for s in 1:nrow(scen_prob)
            @constraint(Mod, int_ex_flow_3[h,s,t] .== (1 ./ int_e(3).reacT[h]) .* (node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID1[h],s,t] .- node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID2[h],s,t]))
            @constraint(Mod, int_ex_flow_3[h,s,t] .<= int_e(3).ptMax[h]./100)
            @constraint(Mod, -int_e(3).ptMax[h]./100 .<= int_ex_flow_3[h,s,t])
        end
    end
end

In [27]:
# Collect the optimization problem and Solve

@objective(Mod, Min, Mod[:total_cost])
optimize!(Mod)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 49824 rows, 20196 columns and 103872 nonzeros
Model fingerprint: 0x35926fff
Variable types: 20160 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [2e-01, 1e+01]
  Bounds range     [6e+00, 6e+00]
  RHS range        [8e-06, 5e+00]
Presolve removed 24768 rows and 4512 columns
Presolve time: 0.17s
Presolved: 25056 rows, 15684 columns, 85728 nonzeros
Variable types: 15648 continuous, 36 integer (36 binary)

Root relaxation: objective 4.924840e+01, 13518 iterations, 0.82 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   49.24840    0   16          -   49.24840      -     -    1s
H    0     0                     243.9770862   49.24840  79.8%     -    1s
 

In [29]:
# Collect the results after the model has completed

Centralized_results = Dict()   
if termination_status(Mod) == MOI.OPTIMAL
    println("Optimal solution found.")
    Centralized_results["shared_line_decision"] = value.(shared_line_decision_var)
    Centralized_results["shared_cand_power"] = value.(shared_cand_flow) .* base_power
    Centralized_results["node_voltage_phase_angle"] = value.(node_voltage_phase_angle) # this could be packaged up more nicely
    Centralized_results["shared_ex_power"] = value.(shared_ex_flow) .* base_power
    Centralized_results["int_line_decision_1"] = value.(int_line_decision_var_1)
    Centralized_results["int_line_decision_2"] = value.(int_line_decision_var_2)
    Centralized_results["int_line_decision_3"] = value.(int_line_decision_var_3)
    Centralized_results["int_line_flow_1"] = value.(int_cand_flow_1) .* base_power
    Centralized_results["int_line_flow_2"] = value.(int_cand_flow_2) .* base_power
    Centralized_results["int_line_flow_3"] = value.(int_cand_flow_3) .* base_power
    Centralized_results["generation_1"] = value.(gen_var_1) .* base_power
    Centralized_results["generation_2"] = value.(gen_var_2) .* base_power
    Centralized_results["generation_3"] = value.(gen_var_3) .* base_power
    Centralized_results["internal_existing_line_flow_1"] = value.(int_ex_flow_1) .* base_power
    Centralized_results["internal_existing_line_flow_2"] = value.(int_ex_flow_2) .* base_power
    Centralized_results["internal_existing_line_flow_3"] = value.(int_ex_flow_3) .* base_power

    Centralized_results["obj_value"] = objective_value(Mod)

    print(Centralized_results)
else
    println("Optimal solution not found.")
    value = JuMP.dual
end



Optimal solution found.
Dict{Any,Any}("node_voltage_phase_angle" =>   [1, 1, 1, 1  ]  =  0.0701253
  [1, 1, 1, 10 ]  =  0.0578224
  [1, 1, 1, 11 ]  =  0.107919
  [1, 1, 1, 12 ]  =  0.0752038
  [1, 1, 1, 13 ]  =  0.0730303
  [1, 1, 1, 14 ]  =  0.0378548
  [1, 1, 1, 15 ]  =  0.00934428
  [1, 1, 1, 16 ]  =  0.0253879
  [1, 1, 1, 17 ]  =  0.0113402
  [1, 1, 1, 18 ]  =  0.0280634
  [1, 1, 1, 19 ]  =  0.111543
  [1, 1, 1, 2  ]  =  0.0442802
  [1, 1, 1, 20 ]  =  0.0474545
  [1, 1, 1, 21 ]  =  0.0418388
  [1, 1, 1, 22 ]  =  0.0143827
  [1, 1, 1, 23 ]  =  0.0684371
  [1, 1, 1, 24 ]  =  0.041307
  [1, 1, 1, 3  ]  =  0.0104564
  [1, 1, 1, 4  ]  =  0.0333714
  [1, 1, 1, 5  ]  =  0.0177587
  [1, 1, 1, 6  ]  =  0.0392611
  [1, 1, 1, 7  ]  =  0.115721
  [1, 1, 1, 8  ]  =  0.0943181
  [1, 1, 1, 9  ]  =  0.0916865
  [1, 1, 2, 1  ]  =  0.0318236
  [1, 1, 2, 10 ]  =  0.0311449
  [1, 1, 2, 11 ]  =  0.0325994
  [1, 1, 2, 12 ]  =  0.0133626
  [1, 1, 2, 13 ]  =  0.0455819
  [1, 1, 2, 14 ]  =  0.0407205
  [1,

In [30]:
Centralized_results["shared_line_decision"]

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0

In [31]:
Centralized_results["shared_cand_power"]

6×4×24 Array{Float64,3}:
[:, :, 1] =
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
 97.4015  43.8448  45.7371  24.7923
 51.1057  28.4598  36.4908  22.186
  0.0      0.0      0.0      0.0

[:, :, 2] =
   0.0     0.0      0.0      0.0
   0.0     0.0      0.0      0.0
   0.0     0.0      0.0      0.0
 100.0    31.154   84.2306  21.8924
  74.597  37.6374  51.363   15.9504
   0.0     0.0      0.0      0.0

[:, :, 3] =
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
 60.6863  49.3835  29.5056  23.8204
 51.4558  43.244   50.1429  12.8626
  0.0      0.0      0.0      0.0

...

[:, :, 22] =
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
 71.2432  36.7076  59.7185  29.7793
 68.3243  29.1328  54.7586  23.4475
  0.0      0.0      0.0      0.0

[:, :, 23] =
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0

In [ ]:
Centralized_results["node_voltage_phase_angle"]

In [32]:
Centralized_results["shared_ex_power"]

8×4×24 Array{Float64,3}:
[:, :, 1] =
  45.6628   19.7269    21.0948    11.9723
 -45.6222  -22.0198   -26.5637   -14.2182
 100.0      44.7       38.8789    20.9916
  37.0552   15.9164    13.6441     7.11425
 -47.5943  -20.8748   -21.8339   -11.1136
  13.3709    7.76245    8.26259    6.25986
  51.7666   32.8001    55.1586    29.0727
 -17.986   -11.0299   -14.8741   -10.011

[:, :, 2] =
  46.0381   12.768     41.0536   10.4268
 -52.2253  -17.8759   -47.3358  -12.1258
  92.8489   18.5622    93.6775   21.5243
  32.9156    5.1048    32.9273    7.45341
 -46.3667  -10.8463   -45.1125  -11.128
  19.0402    9.77988   14.7481    4.64476
 100.0      55.4832    59.2714   20.063
 -33.6983  -16.6589   -18.5279   -6.61013

[:, :, 3] =
  27.4985   23.5621   14.2053     11.0299
 -33.8171  -27.5917  -22.8109    -12.6671
  44.2108   44.5542    8.56762    25.5201
  14.8043   15.3157    0.996899    9.41305
 -22.0569  -19.83     -9.81855   -13.7659
  12.1558   12.7507   14.6461      3.61696
  65.3926   59.30

In [33]:
Centralized_results["int_line_decision_1"]

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0

In [34]:
Centralized_results["int_line_decision_2"]

18-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [35]:
Centralized_results["int_line_decision_3"]

2-element Array{Float64,1}:
 0.0
 0.0

In [36]:
Centralized_results["int_line_flow_1"]

10×4×24 Array{Float64,3}:
[:, :, 1] =
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
 60.0641  26.0675  23.7917  14.1415

[:, :, 2] =
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
 58.7216  14.8601  51.8109  12.795

[:, :, 3] =
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0
  0.0  

In [37]:
Centralized_results["int_line_flow_2"]

18×4×24 Array{Float64,3}:
[:, :, 1] =
 -59.3463  -38.8882  -49.3712  -27.9992
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
 -74.2586  -30.8455  -33.147   -17.9912
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
 -24.9607  -11.2941  -12.295    -6.55124
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0

[:, :, 2] =
 -95.9438  -47.3546   -71.1267  -20.6518
   0.0       0.0        0.0       0.0
   0.0       0.0        0.0       0.0
   0.0       0.0        0.0       0.0
   0.0       0.0        0.0       0.0
 -71.9444  -22.7509   -60.2392  -15.8249
   0.0       0.0        0.0       0.0
 

In [38]:
Centralized_results["int_line_flow_3"]

2×4×24 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

...

[:, :, 22] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 23] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 24] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [39]:
Centralized_results["generation_1"]

14×4×24 Array{Float64,3}:
[:, :, 1] =
 3.09415  0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0
 0.0      0.0  0.0  0.0

[:, :, 2] =
 0.580855  0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0
 0.0       0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0 

In [40]:
Centralized_results["generation_2"]

30×4×24 Array{Float64,3}:
[:, :, 1] =
  0.0      0.0      0.0    0.0
 43.0422  28.8502  61.27  17.4382
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
 40.0      0.0      0.0    0.0
  ⋮                       
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0
  0.0      0.0      0.0    0.0

[:, :, 2] =
  0.708744   0.0      0.0      0.0
 80.0       50.9055  75.7206  19.6965
  0.0        0.0      0.0      0.0
  0.0        0.0      0.0      0.0
 

In [41]:
Centralized_results["generation_3"]

5×4×24 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

...

[:, :, 22] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 23] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 24] =
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [42]:
Centralized_results["internal_existing_line_flow_1"]

20×4×24 Array{Float64,3}:
[:, :, 1] =
  22.9263      8.14373      5.2659       4.32268
  -7.36235    -2.95106     -3.30813     -1.29484
  28.5699     13.6409      -1.31854      2.889
  -5.07332    -1.21073     -2.78769     -0.598577
 -17.2455     -6.55664     -6.03536     -3.13189
 -38.3003    -17.0378      -1.34768     -3.96116
 -49.9673    -22.0041     -13.2487     -10.4259
 -43.5092    -19.7567     -13.583       -9.3764
 -15.9332     -7.43427     -3.62134     -3.012
 -40.078     -17.8254     -13.6667      -8.17034
   0.252798   -0.194143     0.0922385   -0.506958
 -24.2623    -11.3234     -10.1514      -5.28771
 -18.6189     -7.59639     -6.75854     -3.58233
   0.0         0.0          0.0          0.0
   2.15366    -0.0297612    7.5118       2.59593
 -41.8065    -19.9473     -23.2364     -12.4342
 -25.7612    -10.3629     -14.3949      -7.73928
   1.2499      1.88876     -0.0325193    0.660972
  18.9167      9.54105      8.58717      4.43256
  -3.43934    -1.51767     -2.9412     

In [43]:
Centralized_results["internal_existing_line_flow_2"]

41×4×24 Array{Float64,3}:
[:, :, 1] =
 -16.5572    -11.941     -20.6514    -8.06371
  16.5572     11.941      20.6514     8.06371
  28.3506     20.1359     33.9487    14.2199
 -20.3318    -12.956     -16.4805   -11.676
  26.5954     16.7448     29.1306    11.4473
  32.6443     22.1202     34.2433    17.789
  -5.69406   -13.1764    -18.3858   -10.9136
   1.3526     10.3408     13.0672     7.09233
  31.4347     19.8441     38.6781    17.3967
 -61.8798    -39.2607    -47.7701   -28.2379
  13.8287      3.04282     6.59785    1.68012
  40.4666     23.3974     28.4087    15.3948
  14.1186      9.02552    10.9928     5.99197
   ⋮                                
  31.1071     12.4877     14.3881     7.41485
 -40.8323    -20.9185    -22.5855   -12.0992
   3.63539     2.46172     7.03344    4.9574
  -5.20711    -3.14661    -2.01636   -1.24128
  13.3618      6.70151     8.01863    4.19204
   3.49816     1.08903     3.94678    1.5774
   1.32065     1.2276      1.82631    0.408247
 -11.1934     -7.

In [44]:
Centralized_results["internal_existing_line_flow_3"]

7×4×24 Array{Float64,3}:
[:, :, 1] =
 -40.9      -19.8547   -19.1436   -10.6683
  -6.69425   -1.0201    -2.69031   -0.445282
   4.70767    5.2581     2.79413    2.9624
   5.60667    5.45081    5.06362    4.23345
 -51.5348   -25.2094   -22.894    -15.0187
   5.39399    1.15629   13.6169     7.62629
 -29.9724   -16.6928   -15.2447   -10.6845

[:, :, 2] =
 -43.078    -11.8651   -40.0854   -10.3235
  -3.28878    1.01873   -5.02712   -0.804514
   9.97429    5.31333    6.65897    2.36847
  12.2992     7.65894    9.27201    3.14852
 -52.4646   -19.0232   -46.8645   -11.7347
  13.9494    14.0737    15.6783     4.68027
 -35.4567   -15.2558   -30.3863    -8.22876

[:, :, 3] =
 -21.8926    -18.8871    -13.1103   -11.925
  -0.164365   -0.942927    3.29175   -1.84085
   7.07837     5.03845     8.75909    1.52055
   8.58645     7.60733    11.1224     1.72626
 -37.9017    -27.5679    -20.9373   -13.5893
   9.04849    15.4133     14.18       1.23426
 -25.3907    -19.4895    -18.8105    -8.08934

...



In [45]:
Centralized_results["obj_value"]

69.9656573458172